# running the w3c validator

TLDR: the python package isn't the uptodate to validator and we use node.

i use https://validator.w3.org/ a lot in my accessibility testing. it always asks if i am about robot now. i thought it would be better to have a local version i could use. this document is how i got it working after significant trouble shooting.

## installation
installing the w3c nu validator https://github.com/validator/validator/ is a good candidate for `conda` because we'll
use python, java, and node.  i tried using python's `html5validator`, but the 
version of the checker that it bundles is old. the best solution for me was to use
`node` cause i can figure it out. 

use a conda `environment.yml` with at least:

```yml
channels:
  - conda-forge
dependencies:
  - python=3.11
  - openjdk
  - nodejs
```

install the jar distributed on `npm`

`npm install -g vnu-jar`

## running the validator

get the path to the jar from the node binary

In [2]:
    import pathlib, pandas, json, subprocess, shlex
    VNU_JAR = pathlib.Path(subprocess.check_output(
        shlex.split(
            "npm root vnu-jar"
        )
    ).strip().decode()) / "vnu-jar/build/dist/vnu.jar"
    assert VNU_JAR.exists()

`validate_html` runs the checker and returns the serialized payload.

In [3]:
    def validate_html(*files: pathlib.Path) -> dict:
        return json.loads(subprocess.check_output(
            shlex.split(
                F"java -jar {VNU_JAR} --stdout --format json --exit-zero-always"
            ) + list(files)
        ).decode())

## explore the data as a `pandas.Dataframe`

In [70]:
    df = pandas.DataFrame(pandas.Series(validate_html(HTML)).messages)
    del df["url"]
    df

,type,lastLine,lastColumn,firstColumn,subType,message,extract,hiliteStart,hiliteLength,firstLine
0,info,81,27,4,warning,Section lacks heading. Consider using “h2”-“h6...,"ader"">\n <section id=""skip-link"">\n <",10,24,NaN
1,error,295,126,11,NaN,The “role” attribute must not be used on a “tr...,"</tr>\n <tr aria-labelledby=""nb-cell-...",10,127,294.0
2,error,296,40,127,NaN,The “role” attribute must not be used on a “td...,"listitem"">\n <td class=""nb-anchor"" role=""...",10,41,295.0
3,error,301,49,12,NaN,The “role” attribute must not be used on a “td...,"</td>\n <td class=""nb-execution_coun...",10,50,300.0
4,error,306,43,12,NaN,The “role” attribute must not be used on a “td...,"</td>\n <td class=""nb-cell_type"" rol...",10,44,305.0
...,...,...,...,...,...,...,...,...,...,...
173,error,1869,54,12,NaN,The “role” attribute must not be used on a “td...,"</td>\n <td class=""nb-end"" id=""cell-...",10,55,1868.0
174,error,1874,40,12,NaN,The “role” attribute must not be used on a “td...,"</td>\n <td class=""nb-source"" role=""...",10,41,1873.0
175,error,1897,42,12,NaN,The “role” attribute must not be used on a “td...,"</td>\n <td class=""nb-metadata"" role...",10,43,1896.0
176,error,1912,54,12,NaN,The “role” attribute must not be used on a “td...,"</td>\n <td class=""nb-loc"" id=""cell-...",10,55,1911.0
